In [1]:
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")
import lightgbm as lgb
from bayes_opt import BayesianOptimization
import sys
sys.path.append("../fraud_detection/src/")
from util import lgb_f1_score

In [2]:
import sys
sys.path.append("../fraud_detection/src/")
import time
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
from contextlib import contextmanager
import gc 
from util import s_to_time_format, string_to_datetime, hour_to_range, kfold_lightgbm, kfold_xgb
from util import rolling_stats_target_by_cols
#from util import _time_elapsed_between_last_transactions,time_elapsed_between_last_transactions
#from util import num_transaction_in_past_n_days
#from util import add_auto_encoder_feature
#from util import group_target_by_cols_split_by_users
from time import strftime, localtime
import logging
import sys
from config import Configs

# logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))
#log_file = '{}-{}-{}.log'.format(opt.model_name, opt.dataset, strftime("%y%m%d-%H%M", localtime()))
log_file = '../fraud_detection/result/fs_{}.log'.format(strftime("%y%m%d-%H%M", localtime()))
logger.addHandler(logging.FileHandler(log_file))

def group_target_by_cols(df_train, df_test, recipe):
    df = pd.concat([df_train, df_test], axis = 0)
    for m in range(len(recipe)):
        cols = recipe[m][0]
        for n in range(len(recipe[m][1])):
            target = recipe[m][1][n][0]
            method = recipe[m][1][n][1]
            name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)
            tmp = df[cols + [target]].groupby(cols).agg(method)
            tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target})
            df_train = df_train.merge(tmp, how='left', on=cols)
            df_test = df_test.merge(tmp, how='left', on=cols)

        # reduced memory    
        del tmp
        gc.collect()
    
    return df_train, df_test

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    logger.info("{} - done in {:.0f}s".format(title, time.time() - t0))
    
def main(args):
    if args.load_feature == True:
        with timer("Load train/test features extracted"):
            #-------------------------
            # load dataset
            #-------------------------
            df_train = pd.read_csv("../fraud_detection/features/train.csv")
            df_test = pd.read_csv("../fraud_detection/features/test.csv")

            #-------------------------
            # pre-processing
            #-------------------------

            for cat in Configs.CATEGORY:
                df_train[cat] = df_train[cat].astype('category') #.cat.codes
                df_test[cat] = df_test[cat].astype('category')
            for df in [df_train, df_test]:
                df["hour_range"] = df["hour_range"].astype('category')

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))     

    else:
        with timer("Process train/test application"):
            #-------------------------
            # load dataset
            #-------------------------
            df_train = pd.read_csv(args.train_file)
            df_test = pd.read_csv(args.test_file)

            #-------------------------
            # pre-processing
            #-------------------------

            for cat in Configs.CATEGORY:
                df_train[cat] = df_train[cat].astype('category') #.cat.codes
                df_test[cat] = df_test[cat].astype('category')
                
            for df in [df_train, df_test]:
                # pre-processing
                df["loctm_"] = df.loctm.astype(int).astype(str)
                df.loctm_ = df.loctm_.apply(s_to_time_format).apply(string_to_datetime)
                # # time-related feature
                df["loctm_hour_of_day"] = df.loctm_.apply(lambda x: x.hour).astype('category')
                df["loctm_minute_of_hour"] = df.loctm_.apply(lambda x: x.minute)
                df["loctm_second_of_min"] = df.loctm_.apply(lambda x: x.second)
                # df["loctm_absolute_time"] = [h*60+m for h,m in zip(df.loctm_hour_of_day,df.loctm_minute_of_hour)]
                df["hour_range"] = df.loctm_.apply(lambda x: hour_to_range(x.hour)).astype("category")
                # removed the columns no need
                df.drop(columns = ["loctm_"], axis = 1, inplace = True)
                # auxiliary fields
                df["day_hr_min"] = ["{}:{}:{}".format(i,j,k) for i,j,k in zip(df.locdt,df.loctm_hour_of_day,df.loctm_minute_of_hour)]
                df["day_hr_min_sec"] = ["{}:{}:{}:{}".format(i,j,k,z) for i,j,k,z in zip(df.locdt,df.loctm_hour_of_day,df.loctm_minute_of_hour,df.loctm_second_of_min)]

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add bacno/cano feature"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.CONAM_AGG_RECIPE_1)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add iterm-related feature"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.ITERM_AGG_RECIPE)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add conam-related feature"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.CONAM_AGG_RECIPE_2)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add hour-related feature"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.HOUR_AGG_RECIPE)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add cano/conam feature"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.CANO_CONAM_COUNT_RECIPE)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add cano/bacno latent feature"):
            df = pd.read_csv("../fraud_detection/features/bacno_latent_features_w_cano.csv")
            df_train = df_train.merge(df, on = "bacno", how = "left")
            df_test = df_test.merge(df, on = "bacno", how = "left")
            df = pd.read_csv("../fraud_detection/features/bacno_cano_latent_features.csv")
            df_train = df_train.merge(df, on = "cano", how = "left")
            df_test = df_test.merge(df, on = "cano", how = "left")

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add locdt-related feature"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.LOCDT_CONAM_RECIPE)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add mchno-related feature"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.MCHNO_CONAM_RECIPE)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add scity-related feature"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.SCITY_CONAM_RECIPE)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add stocn-related feature"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.STOCN_CONAM_RECIPE)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add mchno/bacno latent feature"):
            df = pd.read_csv("../fraud_detection/features/bacno_latent_features_w_mchno.csv")
            df_train = df_train.merge(df, on = "bacno", how = "left")
            df_test = df_test.merge(df, on = "bacno", how = "left")
            df = pd.read_csv("../fraud_detection/features/bacno_mchno_latent_features.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add time second-level feature on bacno"):
            df_train, df_test = group_target_by_cols(
                df_train, 
                df_test, 
                Configs.HOUR_AGG_SEC_LEVEL_RECIPE_BACNO,
                )
            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add time second-level feature on cano"):
            df_train, df_test = group_target_by_cols(
                df_train, 
                df_test, 
                Configs.HOUR_AGG_SEC_LEVEL_RECIPE_CANO,
                )
            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add time second-level feature on mchno"):
            df_train, df_test = group_target_by_cols(
                df_train, 
                df_test, 
                Configs.HOUR_AGG_SEC_LEVEL_RECIPE_MCHNO,
                )
            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add time second-level feature on csmcu/stocn/scity"):
            df_train, df_test = group_target_by_cols(
                df_train, 
                df_test, 
                Configs.HOUR_AGG_SEC_LEVEL_RECIPE,
                )
            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add time second-level feature on acqic/csmcu/stocn/scity"):
            df_train, df_test = group_target_by_cols(
                df_train, 
                df_test, 
                Configs.HOUR_AGG_SEC_LEVEL_RECIPE_2,
                )
            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add conam-related feature v3"):
            df_train, df_test = group_target_by_cols(
                df_train, 
                df_test, 
                Configs.CONAM_AGG_RECIPE_3,
                )
            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add locdt-related feature v2"):
            df_train, df_test = group_target_by_cols(df_train, df_test, Configs.LOCDT_CONAM_RECIPE_2)

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add conam-related feature v4"):
            df_train, df_test = group_target_by_cols(
                df_train, 
                df_test, 
                Configs.CONAM_AGG_RECIPE_4,
                )
            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add cano/mchno latent feature"):
            df = pd.read_csv("../fraud_detection/features/cano_latent_features_w_mchno.csv")
            df_train = df_train.merge(df, on = "cano", how = "left")
            df_test = df_test.merge(df, on = "cano", how = "left")
            df = pd.read_csv("../fraud_detection/features/cano_mchno_latent_features.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add cano/locdt latent feature"):
            df = pd.read_csv("../fraud_detection/features/cano_latent_features_w_locdt.csv")
            df_train = df_train.merge(df, on = "cano", how = "left")
            df_test = df_test.merge(df, on = "cano", how = "left")
            df = pd.read_csv("../fraud_detection/features/cano_locdt_latent_features.csv")
            df_train = df_train.merge(df, on = "locdt", how = "left")
            df_test = df_test.merge(df, on = "locdt", how = "left")

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add mchno/locdt latent feature"):
            df = pd.read_csv("../fraud_detection/features/mchno_latent_features_w_locdt.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")
            df = pd.read_csv("../fraud_detection/features/mchno_locdt_latent_features.csv")
            df_train = df_train.merge(df, on = "locdt", how = "left")
            df_test = df_test.merge(df, on = "locdt", how = "left")

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

        with timer("Add mchno time aggregate average feature"):
            # df = pd.read_csv("../features/average_mchno_time_agg.csv")
            # df_train = df_train.merge(df, on = "txkey", how = "left")
            # df_test = df_test.merge(df, on = "txkey", how = "left")
            df = pd.read_csv("../fraud_detection/features/average_mchno_mean_conam_in_past_7_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            df = pd.read_csv("../fraud_detection/features/average_mchno_mean_conam_in_past_14_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            df = pd.read_csv("../fraud_detection/features/average_mchno_std_conam_in_past_7_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            df = pd.read_csv("../fraud_detection/features/average_mchno_std_conam_in_past_14_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            df = pd.read_csv("../fraud_detection/features/average_mchno_min_conam_in_past_7_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            df = pd.read_csv("../fraud_detection/features/average_mchno_min_conam_in_past_14_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            df = pd.read_csv("../fraud_detection/features/average_mchno_max_conam_in_past_7_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            df = pd.read_csv("../fraud_detection/features/average_mchno_max_conam_in_past_14_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            df = pd.read_csv("../fraud_detection/features/average_mchno_median_conam_in_past_7_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            df = pd.read_csv("../fraud_detection/features/average_mchno_median_conam_in_past_14_days.csv")
            df_train = df_train.merge(df, on = "mchno", how = "left")
            df_test = df_test.merge(df, on = "mchno", how = "left")

            logger.info("Train application df shape: {}".format(df_train.shape))
            logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add bacno time aggregate average feature"):
        df = pd.read_csv("../fraud_detection/features/average_bacno_min_conam_in_past_7_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        df = pd.read_csv("../fraud_detection/features/average_bacno_max_conam_in_past_7_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        df = pd.read_csv("../fraud_detection/features/average_bacno_mean_conam_in_past_7_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        df = pd.read_csv("../fraud_detection/features/average_bacno_median_conam_in_past_7_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        df = pd.read_csv("../fraud_detection/features/average_bacno_std_conam_in_past_7_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        df = pd.read_csv("../fraud_detection/features/average_bacno_min_conam_in_past_14_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        df = pd.read_csv("../fraud_detection/features/average_bacno_max_conam_in_past_14_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        df = pd.read_csv("../fraud_detection/features/average_bacno_mean_conam_in_past_14_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        df = pd.read_csv("../fraud_detection/features/average_bacno_median_conam_in_past_14_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        df = pd.read_csv("../fraud_detection/features/average_bacno_std_conam_in_past_14_days.csv").iloc[:,1:]
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add mcc time aggregate average feature"):
#         df = pd.read_csv("../fraud_detection/features/average_mcc_median_conam_in_past_7_days.csv")
#         df_train = df_train.merge(df, on = "mcc", how = "left")
#         df_test = df_test.merge(df, on = "mcc", how = "left")

#         df = pd.read_csv("../fraud_detection/features/average_mcc_max_conam_in_past_7_days.csv")
#         df_train = df_train.merge(df, on = "mcc", how = "left")
#         df_test = df_test.merge(df, on = "mcc", how = "left")

#         df = pd.read_csv("../fraud_detection/features/average_mcc_min_conam_in_past_7_days.csv")
#         df_train = df_train.merge(df, on = "mcc", how = "left")
#         df_test = df_test.merge(df, on = "mcc", how = "left")

#         df = pd.read_csv("../fraud_detection/features/average_mcc_mean_conam_in_past_7_days.csv")
#         df_train = df_train.merge(df, on = "mcc", how = "left")
#         df_test = df_test.merge(df, on = "mcc", how = "left")

#         df = pd.read_csv("../fraud_detection/features/average_mcc_std_conam_in_past_7_days.csv")
#         df_train = df_train.merge(df, on = "mcc", how = "left")
#         df_test = df_test.merge(df, on = "mcc", how = "left")

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add scity time aggregate feature"):
#         df = pd.read_csv("../fraud_detection/features/scity_mean_conam_in_past_7_days.csv")
#         df_train = df_train.merge(df, on = ["scity","locdt"], how = "left")
#         df_test = df_test.merge(df, on = ["scity","locdt"], how = "left")

#         df = pd.read_csv("../fraud_detection/features/scity_mean_conam_in_past_14_days.csv")
#         df_train = df_train.merge(df, on = ["scity","locdt"], how = "left")
#         df_test = df_test.merge(df, on = ["scity","locdt"], how = "left")

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add stocn time aggregate feature"):
#         df = pd.read_csv("../fraud_detection/features/stocn_mean_conam_in_past_7_days.csv")
#         df_train = df_train.merge(df, on = ["stocn","locdt"], how = "left")
#         df_test = df_test.merge(df, on = ["stocn","locdt"], how = "left")

#         df = pd.read_csv("../fraud_detection/features/stocn_mean_conam_in_past_14_days.csv")
#         df_train = df_train.merge(df, on = ["stocn","locdt"], how = "left")
#         df_test = df_test.merge(df, on = ["stocn","locdt"], how = "left")
        
#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add acqic time aggregate feature"):
#         df = pd.read_csv("../fraud_detection/features/acqic_mean_conam_in_past_7_days.csv")
#         df_train = df_train.merge(df, on = ["acqic","locdt"], how = "left")
#         df_test = df_test.merge(df, on = ["acqic","locdt"], how = "left")

#         df = pd.read_csv("../fraud_detection/features/acqic_mean_conam_in_past_14_days.csv")
#         df_train = df_train.merge(df, on = ["acqic","locdt"], how = "left")
#         df_test = df_test.merge(df, on = ["acqic","locdt"], how = "left")
        
#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add mchno time aggregate feature"):
#         df = pd.read_csv("../fraud_detection/features/mchno_mean_conam_in_past_7_days.csv")
#         df_train = df_train.merge(df, on = ["mchno","locdt"], how = "left")
#         df_test = df_test.merge(df, on = ["mchno","locdt"], how = "left")

#         df = pd.read_csv("../fraud_detection/features/mchno_mean_conam_in_past_14_days.csv")
#         df_train = df_train.merge(df, on = ["mchno","locdt"], how = "left")
#         df_test = df_test.merge(df, on = ["mchno","locdt"], how = "left")
        
#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))   
        
    return df_train, df_test

args = {
 "train_file":"/data/yunrui_li/fraud/dataset/train.csv",
 "test_file":"/data/yunrui_li/fraud/dataset/test.csv",
 "result_path":"/data/yunrui_li/fraud/fraud_detection/result/submission.csv",
 "feature_selection":True,
 "feature_importance_plot": False,
 "SEED": 1030,
 "NUM_FOLDS": 5, # 5
 "CPU_USE_RATE":1.0,
 "STRATIFIED": True,
 "NUM_LEAVES":31,
 "COLSAMPLE_BYTREE":1.0,
 "SUBSAMPLE": 1.0,
 "SUBSAMPLE_FREQ": 0,
 "MAX_DEPTH": -1,
 "REG_ALPHA": 0.0,
 "REG_LAMBDA": 0.0,
 "MIN_SPLIT_GAIN": 0.0,
 "MIN_CHILD_WEIGHT": 0.001,
 "MAX_BIN": 255,
 "SCALE_POS_WEIGHT": 3,
 "TEST_NULL_HYPO":False,
 "model": "lgb",
 "ensemble":False,
 "seed":1030,
 "load_feature": True
}

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
args = AttrDict(args)
df_train, df_test = main(args)


Train application df shape: (1521787, 477)
Test application df shape: (421665, 476)
Load train/test features extracted - done in 124s
Train application df shape: (1521787, 487)
Test application df shape: (421665, 486)
Add bacno time aggregate average feature - done in 236s


In [3]:
for df in [df_train, df_test]:
    # drop random features (by null hypothesis)
    df.drop(Configs.FEATURE_GRAVEYARD, axis=1, inplace=True, errors='ignore')

    # drop unused features features_with_no_imp_at_least_twice
    df.drop(Configs.FEATURE_USELESSNESS, axis=1, inplace=True, errors='ignore')


In [4]:
feats = [f for f in df_train.columns if f not in ["fraud_ind"]]
X,y = df_train[feats], df_train.fraud_ind

In [5]:
X.shape,y.shape

((1521787, 479), (1521787,))

In [8]:
from sklearn.metrics import f1_score

def lgb_f1_score(y_pred, y_true):
    """evaluation metric"""
    #print ("y_pred",y_pred)
    #print ("y_true",y_true)
    y_hat = np.round(y_pred)
    return 'f1', f1_score(y_true.get_label(), y_hat), True

def bayes_parameter_opt_lgb(X, y, 
                            init_round=15, 
                            opt_round=25, 
                            n_folds=5, 
                            random_seed=1030,
                            n_estimators=10000,
                            learning_rate=0.05, 
                            output_process=True):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, categorical_feature='auto', free_raw_data = False)
    # parameters
    def lgb_eval(
        #num_leaves, 
        feature_fraction, bagging_fraction,
                 #max_depth, 
                 #lambda_l1, lambda_l2, min_split_gain, 
                 #min_child_weight, max_bin, 
                 scale_pos_weight):
        params = {'application':'binary',
                  'num_iterations': n_estimators, 
                  'learning_rate':learning_rate, 
                  'early_stopping_round':100, 
                  'n_jobs':-1,
                  }
#         params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        #params['max_depth'] = int(round(max_depth))
#         params['lambda_l1'] = max(lambda_l1, 0)
#         params['lambda_l2'] = max(lambda_l2, 0)
#         params['min_split_gain'] = min_split_gain
#         params['min_child_weight'] = min_child_weight
#         params['max_bin'] = int(round(max_bin))
        params['scale_pos_weight'] = scale_pos_weight

        cv_result = lgb.cv(params, 
                           train_data, 
                           nfold=n_folds,
                           seed=random_seed, 
                           stratified=True, 
                           categorical_feature = "auto",
                           feval=lgb_f1_score)
        print (cv_result)
        return max(cv_result['f1-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {
#         'num_leaves': (24, 45),
                                            'feature_fraction': (0.7, 1.0),
                                            'bagging_fraction': (0.7, 1.0),
#                                             'lambda_l1': (0, 5),
#                                             'lambda_l2': (0, 3),
#                                             'min_split_gain': (0.0, 0.1),
#                                             'min_child_weight': (1, 50),
                                            'scale_pos_weight': (1, 10),
#                                             'max_bin': (255,355),
                                           }, 
                                 random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: 
        pd.DataFrame(lgbBO.res).sort_values(by = "target", ascending=False).to_csv("../fraud_detection/result/bayes_opt_result.csv")
    return lgbBO.max["target"], lgbBO.max["params"] # best score and best parameter
#     return lgbBO
#     # return best parameters
#     return lgbBO.res['max']['max_params']

opt_score, opt_params = bayes_parameter_opt_lgb(X, y, 
                                     init_round=10, 
                                     opt_round=15, 
                                     n_folds=5, 
                                     random_seed=1030, 
                                     n_estimators=10000, 
                                     learning_rate=0.2)

|   iter    |  target   | baggin... | featur... | scale_... |
-------------------------------------------------------------
{'binary_logloss-mean': [0.13200305084554537, 0.1307906566073484, 0.12166192684724164, 0.12632001534631138, 0.11418440268295693, 0.11211154468037596, 0.10940964765671739, 0.10761125451023013, 0.10757863139527961, 0.10590601494524292, 0.1042995945407261, 0.10446240848229124, 0.10411454306724317, 0.10258792476327812], 'binary_logloss-stdv': [0.001780942368420147, 0.0071818964074741665, 0.0081894206406047, 0.020090952517396878, 0.011714045999349908, 0.009456610819359043, 0.009403172049261447, 0.009154416649585152, 0.009582605574448708, 0.009571413624047146, 0.008895079159472422, 0.009432073010574438, 0.008683230372629388, 0.006993704586171763], 'f1-mean': [0.4591369804474138, 0.45474197314869685, 0.48807263074473906, 0.4937311281949549, 0.5284759384269575, 0.5395342406559379, 0.5424374332453109, 0.544653020910084, 0.5451824656496458, 0.5515105806544852, 0.55673606629

{'binary_logloss-mean': [0.15356786946498957, 0.1548232446872609, 0.14175949379802322, 0.139705409234453, 0.12906370457927854, 0.12498516228020842, 0.12150404560549832, 0.11940711363110024, 0.12460308323037732, 0.11979425490296935, 0.11676033946141251, 0.11662161672958675, 0.11607351129188133, 0.11632210312683536, 0.11628347083795551, 0.11508601066520023], 'binary_logloss-stdv': [0.002664980625136717, 0.01638185983484908, 0.009197371134259941, 0.014079730552809998, 0.010932653674583343, 0.012293462619323133, 0.011576063474246265, 0.011959579269350339, 0.015329954367691575, 0.010386588353150017, 0.00999101181394272, 0.010189939216715864, 0.009578726404631324, 0.009646006854327158, 0.009056416269273728, 0.007806896286281986], 'f1-mean': [0.46692127483013524, 0.440957288484275, 0.4792236258646848, 0.4798887475399325, 0.49148824854310114, 0.4925273100892066, 0.4961304199125342, 0.49790565205614107, 0.49520135206454113, 0.5017253370227758, 0.5083572051018967, 0.5137700690609666, 0.517640027

{'binary_logloss-mean': [0.047483078875506936, 0.04832262382491963, 0.046302353366588604, 0.044635574654205055], 'binary_logloss-stdv': [0.00092809175338417, 0.003596941539421229, 0.0023262290450201378, 0.002113050837954337], 'f1-mean': [0.5866811471589426, 0.614284892278231, 0.6260123935082563, 0.6318176563319053], 'f1-stdv': [0.013887586143881787, 0.02466391515127404, 0.017964742226144616, 0.01782062692435651]}
|  22       |  0.6318   |  1.0      |  0.7      |  1.0      |
{'binary_logloss-mean': [0.047483079117517306, 0.04832262395993847, 0.04630235349026526, 0.044635574777946836], 'binary_logloss-stdv': [0.0009280916939960013, 0.0035969415802208156, 0.0023262290224775033, 0.0021130507489797667], 'f1-mean': [0.5866811471589426, 0.614284892278231, 0.6260123935082563, 0.6318176563319053], 'f1-stdv': [0.013887586143881787, 0.02466391515127404, 0.017964742226144616, 0.01782062692435651]}
|  23       |  0.6318   |  0.7      |  0.7      |  1.0      |
{'binary_logloss-mean': [0.047483079477

In [11]:
opt_params

{'bagging_fraction': 0.9999999999997958,
 'feature_fraction': 0.7,
 'scale_pos_weight': 1.0}

In [12]:
opt_score

0.6318176563319053

In [ ]:
opt_params.max["target"], opt_params.max["params"]

In [ ]:
pd.options.display.max_colwidth = 1000
pd.DataFrame(opt_params.res).sort_values(by = "target", ascending=False)

In [ ]:
df = pd.DataFrame({"A":[1,2,3,4],"B":[1,1,1,1]})
df

In [ ]:
df["A"] = df["A"].copy().sample(frac = 1.0).values

In [ ]:
df

In [ ]:
 df.A.copy().sample(frac = 1.0).values

In [ ]:
df

In [ ]:
clf = lgb.LGBMClassifier(
    n_jobs = 3,
    boosting_type = "rf",
    # nthread=int(multiprocessing.cpu_count()*args.CPU_USE_RATE),
    n_estimators=10000,
    )
clf

In [ ]:
help(lgb.cv)